In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [6]:
DF = pd.read_excel('2000 Bancos 1.XLSX')
df_d = pd.read_excel('Descripcion BCI.xlsx').drop(0)
df_d = df_d.dropna(subset=['Saldo'])
######### 
#esta parte solo se aplica cuando es necesario trasformar
#a numero decimal y se tienen separadores de miles
if str(df_d['Saldo'].dtype) == 'object':
    df_d['Saldo'] = df_d['Saldo'].str.replace('.','')
    df_d['Saldo'] = df_d['Saldo'].str.replace(',','.')
    df_d['Saldo'] = df_d['Saldo'].astype(float)
#########
df_d['Saldo'] *= -1
condicion1 = DF['Clase de documento'] == 'CT'
condicion2 = DF['Cuenta de mayor'] == 1101021011
df = DF[condicion1&condicion2]
lista = ['FECHA', 'SUCURSAL', 'N° DE']
df_d = df_d.drop(lista,axis=1)
tabla = df.merge(df_d,how='left',left_on='Importe en moneda doc.', right_on='Saldo')
#########
#problema en el valor -70781, existen 6 valores provenientes del archivo descripcion
#soluciones posibles?, una solucion a esto puede ser descartar valores en que la informacion no es precisa
importes_problematicos = tabla['Importe en moneda doc.'].value_counts() > 1
importes_problematicos = pd.DataFrame(importes_problematicos)
importes_problematicos = importes_problematicos[importes_problematicos['Importe en moneda doc.']]
importes_problematicos

,Importe en moneda doc.
-1.000,True
-100000.000,True
-1178.000,True
-139686.000,True
-45582.000,True
-1179.000,True
-10.000,True
-25317.000,True
-227195.000,True
-13555.000,True


In [7]:
if importes_problematicos.shape[0] > 0:
    importes_problematicos = importes_problematicos.index
    #en la columna 'Importe en moneda doc.' se toman todos los registros que sean parte de los importes_a_eliminar
    condicion = df['Importe en moneda doc.'].isin(importes_problematicos)
    df = df.drop(df[condicion].index)
    condicion = df_d['Saldo'].isin(importes_problematicos)
    df_d = df_d.drop(df_d[condicion].index)
    tabla = df.merge(df_d,how='left',left_on='Importe en moneda doc.', right_on='Saldo')
##########
#Cambiando Texto por descripcion
inter = tabla.dropna(subset=['DESCRIPCION'])
tabla.loc[inter.index,'Texto'] = inter['DESCRIPCION']
tabla = tabla.drop(['DESCRIPCION','Saldo'],axis=1)
#########
#Reconstruyendo
#~ negacion de conjunto boleano
condicion3 = ~DF['Importe en moneda doc.'].isin(importes_problematicos)
tabla.index = DF[condicion1&condicion2&condicion3].index
DF.loc[condicion1&condicion2&condicion3] = tabla
DF.to_excel('2000 Bancos.xlsx',index=False)

In [ ]:
tabla

,Icono part.abiertas/comp.,Ejercicio / mes,Cuenta de mayor,Asignación,Referencia,Nº documento,División,Clase de documento,Fe.contabilización,Fecha valor,Clave contabiliz.,Importe en moneda local,Moneda local,Indicador impuestos,Doc.compensación,Importe en moneda doc.,Moneda del documento,Texto,Nombre del usuario
2332,NaN,2022/03,1101021011.000,0000000,BC11P0001 22062,100113290.000,NaN,CT,2022-03-29,2022-03-29,50.000,-159585,CLP,NaN,NaN,-159585.000,CLP,ABONO CUENTA BCI,VGAMBOA
2333,NaN,2019/06,1101021011.000,0000000,BC11P0001 19106,100127146.000,NaN,CT,2019-06-03,2019-06-03,50.000,-52336,CLP,NaN,NaN,-52336.000,CLP,4K,MQUIROZ
2334,NaN,2020/06,1101021011.000,0000000,BC11P0001 20035,100239184.000,NaN,CT,2020-06-19,2020-06-19,50.000,-377282,CLP,NaN,NaN,-377282.000,CLP,4K,MQUIROZ
2335,NaN,2021/07,1101021011.000,0000000,BC11P0001 21145,100269972.000,NaN,CT,2021-07-30,2021-07-30,50.000,-21063,CLP,NaN,NaN,-21063.000,CLP,PAGO RECIBIDO,AJARA
2337,NaN,2019/05,1101021011.000,201905156295,BC11P0001 19094,100106882.000,NaN,CT,2019-05-15,2019-05-15,50.000,-6295,CLP,NaN,NaN,-6295.000,CLP,1Z,MQUIROZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3963,NaN,2022/06,1101021011.000,20220610951611,BC11P0001 22114,100227197.000,NaN,CT,2022-06-10,2022-06-10,50.000,-951611,CLP,NaN,NaN,-951611.000,CLP,ABONO PNOL 65149672-1 COLEGIO EL BEL,CSEGURA
3965,NaN,2022/06,1101021011.000,2022061095840,BC11P0001 22114,100226424.000,NaN,CT,2022-06-10,2022-06-10,50.000,-95840,CLP,NaN,NaN,-95840.000,CLP,PAGO RECIBIDO MISMO BCO PMS 065158421-3,CSEGURA
3967,NaN,2022/06,1101021011.000,2022061096323,BC11P0001 22114,100226671.000,NaN,CT,2022-06-10,2022-06-10,50.000,-96323,CLP,NaN,NaN,-96323.000,CLP,PAGO RECIBIDO MISMO BCO PMS 076139522-K,CSEGURA
3969,NaN,2022/06,1101021011.000,2022061096390,BC11P0001 22114,100226781.000,NaN,CT,2022-06-10,2022-06-10,50.000,-96390,CLP,NaN,NaN,-96390.000,CLP,PAGO RECIBIDO PRV 061103006-1 BCO ESTAD,CSEGURA


In [ ]:
tabla['Importe en moneda doc.'].value_counts().sort_values(ascending=False)

-159585.000    1
-2437.000      1
-21063.000     1
-6295.000      1
-11240.000     1
              ..
-951611.000    1
-95840.000     1
-96323.000     1
-96390.000     1
-98131.000     1
Name: Importe en moneda doc., Length: 1214, dtype: int64